In [1]:
import os

In [2]:
%pwd

'd:\\Machine_learning\\ETE_Ml_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Machine_learning\\ETE_Ml_project'

# For Entity
**It is a return type**

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) # decorator 
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

# For Configuration

In [11]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 param_filepath=PARAM_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


# For Component

In [13]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [24]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists and size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip:
            zip.extractall(unzip_path)

# For pipeline

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2025-05-21 14:37:49,429 : INFO : common : yaml file: config\config.yaml loaded successfully
2025-05-21 14:37:49,431 : INFO : common : yaml file: params.yaml loaded successfully
2025-05-21 14:37:49,435 : INFO : common : yaml file: schema.yaml loaded successfully
2025-05-21 14:37:49,437 : INFO : common : created directory at: artifacts
2025-05-21 14:37:49,440 : INFO : common : created directory at: artifacts/data_ingestion
2025-05-21 14:37:50,743 : INFO : 3715758236 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d9fccfa514183dfc3e92042446614fd698dbaee53a73a620815683d730cd5483"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A92A:2B730B:507DC5:596909:682D90DC
Accept-Ranges: bytes
Date: 